In [1]:
import datetime
import pandas as pd
import time
import requests
import os
import numpy as np
import xlsxwriter
from abc import ABCMeta, abstractmethod
from Queue import Queue
from collections import deque
import getpass
import pandas.io.formats.excel

### Simulation runner

In [90]:

class SimulationEnv(object):
    
    loggables = ['positions']
    
    def __init__(self, wealth, assets, 
                 path_to_data, start_date, end_date,
                 agent_type, expert_type, reinvest, max_assets=100):

        self.init_wealth = wealth
        self.wealth = wealth
        self.assets = assets
        self.agent_type = agent_type
        self.expert_type = expert_type
        self.path_to_data = path_to_data
        self.reinvest = reinvest
        self.max_assets = max_assets
        self.start_date = start_date
        self.end_date = end_date
        
        return
    
    ## To be called before running a simulation - initialize experts, agents, and initial position, etc.
    def setup_params(self, agent_args={}, expert_args={}):
    
        stdevs = {}
        for f in os.listdir(self.path_to_data):
            df = pd.read_csv(self.path_to_data+f)
            df = df[df["date"] < self.end_date]
            df["return"] = (df["adj_close"] - df["adj_close"].shift(1))/df["adj_close"].shift(1)
            stdevs[f] = df["return"].std()
            self.assets = [f.split(".")[0].upper() for f in sorted(stdevs, key=stdevs.get, reverse=True)[:self.max_assets]]
        
        print self.assets
        
        self.experts = [self.expert_type(a, self.path_to_data, self.start_date, self.end_date, **expert_args) for a in self.assets]
        self.agent = self.agent_type(self.experts, **agent_args)
        self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
        
        return
        
    ## Run simulation
    def run(self, log=False, logpath=os.getcwd()):
        
        ## Start period counter and log
        self.period = 1
        self.finallog = []
        self.log = log
        self.logpath = logpath
        
        ## Warmup period: 
        ## i.e. for strategies involving moving average indicators, wait until we have enough data to calculate MA
        for expert in self.agent.experts:
            expert.warmup()
        
        ## Simulation: Go until data iterators reach the end
        while True:
            try:
                print "PERIOD {}".format(self.period)
                print "dates:"
                print np.array([e.current_date for e in self.agent.experts])
                print "positions:"
                print np.array([weight * self.wealth for weight in self.agent.weights])
                print "weights:"
                print np.array([weight for weight in self.agent.weights])
                
                if self.period > 1:
                    print "rewards:"
                    print np.array([r for r in self.agent.rewards])
                    print "returns:"
                    print np.array([r for r in self.agent.returns])

                print "---------------------"
                ## Log this period
                if log:
                    self.logperiod()
                    
                ## Update experts with last period's rewards
                for i, expert in enumerate(self.agent.experts):
                    expert.update()
                    if self.reinvest:# below is to redistribute unallocated money
                        if expert.pick == False:
                            extra_money = self.positions[i]
                            self.positions[i] = 0
                            distribute = np.greater(self.positions, 0)
                            if not np.any(distribute): # this is to avoid divide by zero if nothing is invested
                                #print("all experts say not to buy")
                                self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
                            else:
                                distribute_extra_money = distribute * (extra_money/ np.sum(distribute))
                                self.positions = np.add(self.positions, distribute_extra_money)
                            assert(abs(np.sum(self.positions)-self.wealth) <= 1)# make sure all money is accounted for
                      
                ## Update agent accordingly (i.e. for Hedge, update weights according to each expert's reward in the last period)
                self.agent.update()
                

                
                ## Update position with returns from last period
                self.positions = self.positions * (1 + self.agent.returns)
                self.wealth = np.sum(self.positions)
                
                ## Rebalance according to new, updated weights
                ## TODO: only allow non-fractional shares to be purchased (?)
                if np.sum(self.agent.weights) == 1:
                    self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
                    
                print "new weights:"
                print np.array([weight for weight in self.agent.weights])
                print "---------------------"
                print "\n\n"
                
                
                ## Advance period
                self.period += 1
                
            except StopIteration:
                break
        
        ## Write the log file
        if log:
            self.savelog()

    def logperiod(self):
        row = [self.period] + [self.wealth]
        nrow = []
        for loggable in (self.loggables):
            if getattr(self,loggable) is None:
                nrow += [None] * len(self.experts)
            else:
                nrow += getattr(self,loggable).tolist()
        arow = []
        for loggable in (self.agent_type.loggables):
            if getattr(self.agent,loggable) is None:
                arow += [None] * len(self.experts)
            else:
                arow += getattr(self.agent,loggable).tolist()
        erow = []
        for loggable in (self.expert_type.loggables):
            if [getattr(e,loggable) for e in self.experts] is None:
                erow += [None] * len(self.experts)
            else:
                erow += [getattr(e,loggable) for e in self.experts]
        row += nrow + arow + erow
        self.finallog.append(row)
    
    def savelog(self):
        runtime = datetime.datetime.now()
        runuser = getpass.getuser()
        logname = runuser + "_" + runtime.strftime('%Y-%m-%d_%H-%M-%S')
        if not os.path.exists(os.path.join(self.logpath, logname)):
            os.makedirs(os.path.join(self.logpath, logname))
        col = ['period', 'wealth'] + \
            [x+'.'+y for x in self.loggables for y in self.assets] + \
            [x+'.'+y for x in self.agent_type.loggables for y in self.assets] + \
            [x+'.'+y for x in self.expert_type.loggables for y in self.assets]
        df = pd.DataFrame(self.finallog, columns=col)
        df.set_index('period', inplace=True)
        
        
        writer = pd.ExcelWriter(os.path.join(self.logpath, logname, logname+'.xlsx'), engine='xlsxwriter')
        pd.io.formats.excel.header_style = None
        df.to_excel(writer,'run_log')
        
        workbook  = writer.book
        worksheet = writer.sheets['run_log']
        
        header_format = workbook.add_format({'bold': True,'text_wrap': True})
        worksheet.set_row(0, None, header_format)
        writer.save()



### Base classes

Base classes for agents and experts to be implemented by us

In [21]:

class Agent(object):
    __metaclass__ =ABCMeta
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
    @abstractmethod
    def act(self):
        pass
    
class Expert(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, reward_data):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
class Context(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, context_data):
        pass
    
    @abstractmethod
    def observe(self):
        pass

### Agents

Implementations of different online portfolio selection algorithms
* Exponential Gradient
* Exponential Gradient (recent history only)

In [26]:
## EG. (http://www.cis.upenn.edu/~mkearns/finread/helmbold98line.pdf)
class EG(Agent):
    
    loggables = ['returns','rewards','weights']
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta):
        self.eta = eta
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = np.asarray([e.returns for e in self.experts])
        multipliers = np.exp(self.eta * self.rewards/np.sum(self.weights * self.rewards))
        self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
        self.weights = np.nan_to_num(self.weights)

        return
    
    def act(self):
        return self.weights


class EGRecent(Agent):
    
    loggables = ['returns','rewards','weights']
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta, n):
        self.eta = eta
        self.experts = experts
        #self.weights = Queue(maxsize=n)
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = deque(maxlen=n)
        self.returns = None
        self.t = 0
        self.n = n
        #self.rewards = None
        #self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
           
        self.rewards.append(np.asarray([e.reward for e in self.experts]))
        self.returns = np.asarray([e.returns for e in self.experts])
        
        self.weights = np.ones(len(self.experts))/len(self.experts)
        #print self.t
        #print len(self.rewards)
        for rewards in self.rewards:
            rewards = np.asarray(rewards)
            multipliers = np.exp(self.eta * rewards/np.sum(self.weights * rewards))
            self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
            self.weights = np.nan_to_num(self.weights)
        self.t += 1
        #print self.t
        
        return
    
    def act(self):
        return self.weights
    
      
        

More agents to serve as naive/benchmark portfolio allocation algorithms:
* Equal-weighted buy and hold
* Market-weighted buy and hold
* Constantly rebalanced portfolio


In [27]:
## Some other examples of agents to use as benchmarks 

class ConstantRebalancer(Agent):
    
    loggables = ['rewards','weights']
    
    def __init__(self, experts):
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = self.rewards - 1.
        return
    
    def act(self):
        return self.weights
    
## TODO
class WeightedBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    

## to be used with "Dummy" Expert for each stock
class NaiveBuyHold(Agent):
    
    def __init__(self, experts):
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = self.rewards - 1.
        self.weights = np.multiply(self.weights, self.rewards)
        self.weights = np.divide(self.weights, np.sum(self.weights))
        assert((np.sum(self.weights) - 1.) <= 0.00001)
        return
    
    def act(self):
        return self.weights

### Experts

* Dummy
* Mean Reversion
* Momentum

In [78]:
data = pd.read_csv("data/djia_20150101_20171101/DD.csv", iterator=True, chunksize=1)



In [80]:
row = data.get_chunk(1)
row = data.get_chunk(1)
datetime.strptime(row["date"].item(), "%Y-%M-%d")

datetime.datetime(2015, 1, 7, 0, 1)

In [92]:

## Dummy expert that always pick the same asset
class Dummy(Expert):
    
    loggables = ['reward']
    
    ## Expert has a reward associated with its pick
    def __init__(self, name, path_to_data, start_date, end_date):
        self.reward = 0.
        self.pick = True ## Might not be necessary
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.current_date = datetime.strptime("1000-01-01", "%Y-%m-%d")
        self.start_date = datetime.strptime(start_date, "%Y-%m-%d")
        self.end_date = datetime.strptime(end_date, "%Y-%m-%d")
        while self.current_date < self.start_date:
            self.last_row = self.data.get_chunk(1)
            self.last_price = float(self.last_row["adj_close"])
            self.current_date = datetime.strptime(self.last_row["date"].item(), "%Y-%m-%d")
        return
    
    ## Expert updates its reward 
    def update(self):
        self.last_row = self.data.get_chunk(1)
        self.current_date = datetime.strptime(self.last_row["date"].item(), "%Y-%m-%d")
        
        if self.current_date > self.end_date:
            raise StopIteration
            
        current_price = float(self.last_row["adj_close"])
        self.reward = current_price/self.last_price
        self.returns = self.reward - 1.
        self.last_price = current_price
        return
    
    def warmup(self):
        pass
    

class MeanReversion(Expert):
    
    loggables = ['avg','std']
    
    def __init__(self, name, path_to_data, window_size, threshold):
        self.reward = 0.
        self.pick = False
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.
        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            self.last_price = float(self.data.get_chunk(1)["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
            
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        current_price = float(self.data.get_chunk(1)["adj_close"])

        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = current_price/self.last_price
            self.returns = self.reward - 1.
        else:
            self.reward = -current_price/self.last_price
            self.returns = 0
        self.last_price = current_price

        if self.last_price <= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return
    
    
class Momentum(Expert):
    
    loggables = ['avg','std']
    
    def __init__(self, name, path_to_data, window_size, threshold):
        self.reward = 0.
        self.pick = False
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.
        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            self.last_price = float(self.data.get_chunk(1)["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
            
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        current_price = float(self.data.get_chunk(1)["adj_close"])

        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = current_price/self.last_price
            self.returns = self.reward - 1.
        else:
            self.reward = -current_price/self.last_price
            self.returns = 0
        self.last_price = current_price

        if self.last_price >= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return
  

### Contexts

In [7]:
def GridSearch(Agent, Expert, reinvest=False, full_data=False, agent_args=[], expert_args=[]):
    ## Run simulation 
    start = time.time()
    if full_data:
        data = "data/djia_20000101_20171101/"
        start_date = "2000-01-01"
        year = 17.
    else:
        data = "data/djia_20150101_20171101/"
        start_date = "2015-01-01"
        year = 2.
    best_wealth = 0
    best_params = {}
    for agent_arguments in agent_args:
        for expert_arguments in expert_args:
            
            s = SimulationEnv(
                initial_wealth, 
                stocks, 
                data, 
                start_date,
                "2017-11-01", 
                Agent, 
                Expert, 
                reinvest
            )
            s.setup_params(
                agent_args=agent_arguments,
                expert_args=expert_arguments
            )
            s.run()

            years = year + 11./12
            ar = ((s.wealth)/initial_wealth)**(1/years) - 1
            end = time.time()

            if s.wealth > best_wealth:
                best_wealth = s.wealth
                best_params = {
                    "agent_args": agent_arguments,
                    "expert_args": expert_arguments,
                    "Initial wealth": initial_wealth,
                    "Final wealth": s.wealth,
                    "Annualized return": ar,
                    "Time": int(end-start),
                }
    print("Best params:", best_params)
    
#GridSearch(ConstantRebalancer, Dummy, reinvest=True, full_data=False, agent_args=[{}], expert_args=[{}])
GridSearch(EG, Dummy, reinvest=True, full_data=False, agent_args=[{"eta":-0.01},{"eta":-0.005}], expert_args=[{}])


NameError: global name 'initial_wealth' is not defined

In [8]:
  
class VolContext(Context):
    
    def __init__(self, data_file, threshold):
        self.data = pd.read_csv(data_file, iterator=True, chunksize=1)
        self.threshold = threshold
        self.contexts = ["HighVol", "LowVol"]
        return
    
    # Returns a string giving the name of the context
    def observe(self):
        if float(self.data.get_chunk(1)["adj_close"]) > self.threshold:
            return self.contexts[0]
        else:
            return self.contexts[1]

In [82]:
stocks = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE',
 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 
'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'WMT', 'XOM']
stocks = [stock.lower() for stock in stocks]

In [83]:
initial_wealth = 1.

In [ ]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(1., stocks, "data/djia_20150101_20171101/", "2016-01-01", "2016-12-31", EG, Dummy, True, max_assets=2)
s.setup_params(agent_args={"eta": 0.01})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

['CAT', 'CVX']
PERIOD 1
dates:
[datetime.datetime(2016, 1, 4, 0, 0) datetime.datetime(2016, 1, 4, 0, 0)]
positions:
[ 0.5  0.5]
weights:
[ 0.5  0.5]
---------------------
new weights:
[ 0.49995246  0.50004754]
---------------------



PERIOD 2
dates:
[datetime.datetime(2016, 1, 5, 0, 0) datetime.datetime(2016, 1, 5, 0, 0)]
positions:
[ 0.49948027  0.49957525]
weights:
[ 0.49995246  0.50004754]
rewards:
[ 0.98955729  1.00855374]
returns:
[-0.01044271  0.00855374]
---------------------
new weights:
[ 0.50001352  0.49998648]
---------------------



PERIOD 3
dates:
[datetime.datetime(2016, 1, 6, 0, 0) datetime.datetime(2016, 1, 6, 0, 0)]
positions:
[ 0.48573849  0.48571221]
weights:
[ 0.50001352  0.49998648]
rewards:
[ 0.98424495  0.96049548]
returns:
[-0.01575505 -0.03950452]
---------------------
new weights:
[ 0.50001613  0.49998387]
---------------------



PERIOD 4
dates:
[datetime.datetime(2016, 1, 7, 0, 0) datetime.datetime(2016, 1, 7, 0, 0)]
positions:
[ 0.4687724   0.46874216]
we

In [40]:
from datetime import datetime


datetime.datetime(2015, 1, 1, 0, 0)

In [13]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(agent_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

['V', 'AAPL', 'HD', 'TRV', 'NKE', 'CAT', 'JNJ', 'UNH', 'DIS', 'PG', 'VZ', 'IBM', 'GE', 'JPM', 'CVX', 'AXP', 'CSCO', 'MSFT', 'XOM', 'GS', 'UTX', 'INTC', 'WMT', 'DD', 'MCD', 'KO', 'MMM', 'MRK', 'PFE', 'BA']
PERIOD 1
positions:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------
new weights:

new weights:
[ 0.03284298  0.03345003  0.03379494  0.03339833  0.0337669   0.03165653
  0.03349374  0.03406244  0.03372804  0.03328206  0.03395968  0.03231368
  0.03203189  0.03151395  0.03148235  0.03230054  0.03400799  0.03335999
  0.03257303  0.03186632  0.03330285  0.03369548  0.03508006  0.03379243
  0.03338479  0.03387171  0.03279712  0.03646552  0.0350131   0.03371152]
---------------------



PERIOD 7
positions:
[ 0.03270445  0.03330894  0.03365239  0.03325746  0.03362447  0.031523
  0.03335247  0.03391877  0.03358578  0.03314168  0.03381644  0.03217738
  0.03189678  0.03138102  0.03134956  0.0321643   0.03386454  0.03321928
  0.03243564  0.03173191  0.03316238  0.03355336  0.03493209  0.03364989
  0.03324398  0.03372884  0.03265878  0.03631171  0.03486541  0.03356932]
weights:
[ 0.03284298  0.03345003  0.03379494  0.03339833  0.0337669   0.03165653
  0.03349374  0.03406244  0.03372804  0.03328206  0.03395968  0.03231368
  0.03203189  0.03151395  0.03148235  0.03230054  0.03400

new weights:
[ 0.03259982  0.0334504   0.03357954  0.03376569  0.03313544  0.03099737
  0.03259856  0.03525356  0.0339933   0.03391703  0.03533004  0.03257742
  0.0320139   0.02998844  0.03181246  0.03164599  0.03419851  0.03337191
  0.03300757  0.03038864  0.03436616  0.03338829  0.03394744  0.03393471
  0.03275078  0.03445229  0.03320341  0.03682602  0.03553848  0.03396682]
---------------------



PERIOD 12
positions:
[ 0.03230448  0.03314735  0.03327532  0.03345979  0.03283525  0.03071655
  0.03230323  0.03493418  0.03368533  0.03360976  0.03500997  0.03228228
  0.03172386  0.02971675  0.03152425  0.03135929  0.03388869  0.03306958
  0.03270854  0.03011333  0.03405482  0.03308581  0.03363989  0.03362728
  0.03245407  0.03414017  0.0329026   0.03649239  0.03521651  0.0336591 ]
weights:
[ 0.03259982  0.0334504   0.03357954  0.03376569  0.03313544  0.03099737
  0.03259856  0.03525356  0.0339933   0.03391703  0.03533004  0.03257742
  0.0320139   0.02998844  0.03181246  0.03164599  0.03

new weights:
[ 0.03190704  0.03620117  0.0347585   0.03404381  0.03370252  0.0301376
  0.03332545  0.03693943  0.03392836  0.03254549  0.03445647  0.03209778
  0.03265278  0.03007243  0.03161945  0.03024466  0.03332302  0.03023516
  0.03251939  0.03055258  0.03488469  0.03188353  0.03469138  0.03339245
  0.03267493  0.03414457  0.03429866  0.03688049  0.03500301  0.03688319]
---------------------



PERIOD 18
positions:
[ 0.03098634  0.03515656  0.03375552  0.03306146  0.03273002  0.02926797
  0.03236382  0.03587352  0.03294933  0.03160637  0.0334622   0.03117158
  0.03171056  0.02920467  0.03070705  0.02937194  0.03236146  0.0293627
  0.03158103  0.02967097  0.03387807  0.03096351  0.03369034  0.03242889
  0.03173208  0.03315931  0.03330895  0.03581628  0.03399298  0.0358189 ]
weights:
[ 0.03190704  0.03620117  0.0347585   0.03404381  0.03370252  0.0301376
  0.03332545  0.03693943  0.03392836  0.03254549  0.03445647  0.03209778
  0.03265278  0.03007243  0.03161945  0.03024466  0.03332

rewards:
[ 1.01963124  1.00766962  1.01091011  1.00650698  0.99132298  0.97652526
  0.9892641   1.00102031  1.07630181  0.99813845  0.99937278  0.99047138
  0.98733143  0.99400564  0.9892267   0.99856682  0.98414454  1.00576923
  0.99143631  0.99189969  0.99074775  1.0041543   1.00533705  1.00559116
  1.00138416  1.00120106  0.99319031  0.9677155   1.01030606  0.99918556]
returns:
[ 0.01963124  0.00766962  0.01091011  0.00650698 -0.00867702 -0.02347474
 -0.0107359   0.00102031  0.07630181 -0.00186155 -0.00062722 -0.00952862
 -0.01266857 -0.00599436 -0.0107733  -0.00143318 -0.01585546  0.00576923
 -0.00856369 -0.00810031 -0.00925225  0.0041543   0.00533705  0.00559116
  0.00138416  0.00120106 -0.00680969 -0.0322845   0.01030606 -0.00081444]
---------------------
new weights:
[ 0.03405291  0.03656846  0.03498836  0.03382383  0.03261298  0.03045113
  0.03254905  0.03585236  0.03635204  0.03205632  0.03464195  0.0323532
  0.03246149  0.0301642   0.03223904  0.03033144  0.03278256  0.030143

...

new weights:
[ 0.03930734  0.03708203  0.03868309  0.03265844  0.03235606  0.03537102
  0.03448967  0.05017443  0.03118949  0.02812511  0.0296604   0.02508584
  0.02865777  0.0402038   0.02694588  0.0242152   0.03195422  0.04290612
  0.02463169  0.03028673  0.02817174  0.02667489  0.02558286  0.03304495
  0.04671801  0.03050182  0.03342098  0.03096913  0.02960643  0.05132488]
---------------------



PERIOD 647
positions:
[ 0.05101232  0.04812436  0.05020217  0.04238351  0.04199108  0.04590383
  0.04476003  0.06511542  0.04047712  0.03650023  0.0384927   0.03255592
  0.0371915   0.05217573  0.03496985  0.03142602  0.04146957  0.05568274
  0.03196654  0.03930554  0.03656075  0.03461817  0.03320095  0.0428851
  0.06062975  0.03958468  0.04337311  0.04019115  0.03842266  0.06660846]
weights:
[ 0.03930734  0.03708203  0.03868309  0.03265844  0.03235606  0.03537102
  0.03448967  0.05017443  0.03118949  0.02812511  0.0296604   0.02508584
  0.02865777  0.0402038   0.02694588  0.0242152   0.03

new weights:
[ 0.03942178  0.03812465  0.03958919  0.03189127  0.03291239  0.03500375
  0.0347507   0.05122615  0.03022274  0.02783488  0.03003301  0.02484511
  0.02835417  0.04071045  0.02767616  0.02457394  0.03185838  0.0421894
  0.02419319  0.03110226  0.02854191  0.02761173  0.02554383  0.03170997
  0.04531884  0.02978454  0.03433646  0.03036897  0.03016284  0.05010734]
---------------------



PERIOD 653
positions:
[ 0.05168859  0.04998783  0.05190809  0.04181482  0.04315369  0.04589581
  0.04556402  0.06716611  0.0396271   0.03649621  0.03937833  0.03257612
  0.03717708  0.05337825  0.0362881   0.03222058  0.0417717   0.05531741
  0.03172134  0.0407803   0.03742325  0.03620363  0.03349226  0.0415771
  0.05942063  0.03905254  0.04502087  0.03981883  0.03954856  0.06569915]
weights:
[ 0.03942178  0.03812465  0.03958919  0.03189127  0.03291239  0.03500375
  0.0347507   0.05122615  0.03022274  0.02783488  0.03003301  0.02484511
  0.02835417  0.04071045  0.02767616  0.02457394  0.031

new weights:
[ 0.0399993   0.03974795  0.040176    0.02924967  0.03308436  0.034978
  0.03499925  0.05116753  0.0285966   0.02827327  0.03009961  0.02473648
  0.02802858  0.04039843  0.02745785  0.02455046  0.03205439  0.04292647
  0.02394253  0.0309238   0.02765686  0.02751741  0.02589972  0.03153083
  0.04655884  0.03012748  0.03445779  0.03030947  0.03001296  0.05053811]
---------------------



PERIOD 659
positions:
[ 0.05219067  0.05186271  0.05242122  0.03816466  0.04316813  0.04563893
  0.04566666  0.06676286  0.03731255  0.03689067  0.03927366  0.0322759
  0.0365714   0.05271145  0.03582671  0.03203318  0.04182424  0.05601002
  0.03123996  0.04034905  0.03608638  0.03590443  0.03379369  0.0411411
  0.0607495   0.03931003  0.04496017  0.03954748  0.0391606   0.0659416 ]
weights:
[ 0.0399993   0.03974795  0.040176    0.02924967  0.03308436  0.034978
  0.03499925  0.05116753  0.0285966   0.02827327  0.03009961  0.02473648
  0.02802858  0.04039843  0.02745785  0.02455046  0.0320543

new weights:
[ 0.04108612  0.03950114  0.03919354  0.0311022   0.0300729   0.03576436
  0.03545526  0.0516511   0.02909466  0.02873527  0.02994926  0.02460324
  0.02729339  0.04014634  0.02724043  0.02459605  0.03160188  0.0430683
  0.02357702  0.03052687  0.02758008  0.02659525  0.02578046  0.03237784
  0.04744562  0.03011152  0.03427199  0.03030491  0.02998391  0.05128909]
---------------------



PERIOD 665
positions:
[ 0.05340293  0.0513428   0.05094299  0.04042603  0.03908817  0.04648581
  0.04608405  0.06713509  0.03781667  0.03734954  0.03892746  0.03197881
  0.03547542  0.05218143  0.03540657  0.03196947  0.0410755   0.05597933
  0.03064495  0.03967822  0.03584805  0.03456798  0.03350894  0.04208408
  0.06166889  0.03913837  0.04454606  0.03938972  0.0389725   0.06666455]
weights:
[ 0.04108612  0.03950114  0.03919354  0.0311022   0.0300729   0.03576436
  0.03545526  0.0516511   0.02909466  0.02873527  0.02994926  0.02460324
  0.02729339  0.04014634  0.02724043  0.02459605  0.03

new weights:
[ 0.04083816  0.04038127  0.03934044  0.03101072  0.02911957  0.03619776
  0.03462344  0.05179588  0.0290421   0.02837135  0.02972401  0.02480461
  0.02686354  0.03992554  0.02751468  0.02463989  0.03223974  0.04344769
  0.02331546  0.03038741  0.02843303  0.02683512  0.02523337  0.03235793
  0.04755985  0.02988961  0.03407882  0.03055965  0.03017124  0.05129812]
---------------------



PERIOD 671
positions:
[ 0.05322846  0.05263295  0.05127634  0.04041938  0.03795445  0.04718017
  0.04512819  0.06751076  0.03785347  0.03697923  0.03874228  0.03233033
  0.03501395  0.05203895  0.03586264  0.03211564  0.04202128  0.05662972
  0.03038938  0.03960695  0.03705961  0.0349769   0.03288917  0.04217534
  0.06198952  0.03895813  0.04441834  0.03983145  0.03932519  0.06686197]
weights:
[ 0.04083816  0.04038127  0.03934044  0.03101072  0.02911957  0.03619776
  0.03462344  0.05179588  0.0290421   0.02837135  0.02972401  0.02480461
  0.02686354  0.03992554  0.02751468  0.02463989  0.0

In [18]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", ConstantRebalancer, Dummy, True)
s.setup_params(agent_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

['V', 'AAPL', 'HD', 'TRV', 'NKE', 'CAT', 'JNJ', 'UNH', 'DIS', 'PG', 'VZ', 'IBM', 'GE', 'JPM', 'CVX', 'AXP', 'CSCO', 'MSFT', 'XOM', 'GS', 'UTX', 'INTC', 'WMT', 'DD', 'MCD', 'KO', 'MMM', 'MRK', 'PFE', 'BA']
PERIOD 1
positions:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------
new weights:

new weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 11
positions:
[ 0.032958  0.032958  0.032958  0.032958  0.032958  0.032958  0.032958
  0.032958  0.032958  0.032958  0.032958  0.032958  0.032958  0.032958
  0.032958  0.032958  0.032958  0.032958  0.032958  0.032958  0.032958
  0.032958  0.032958  0.032958  0.032958  0.032958  0.032958  0.032958
  0.032958  0.032958]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.0

new weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 16
positions:
[ 0.03332766  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766
  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766
  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766
  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766
  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766  0.03332766]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03

new weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 22
positions:
[ 0.03308923  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923
  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923
  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923
  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923
  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923  0.03308923]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03

new weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 28
positions:
[ 0.03358482  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482
  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482
  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482
  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482
  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482  0.03358482]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03

...

new weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 646
positions:
[ 0.04349674  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674
  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674
  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674
  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674
  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674  0.04349674]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.0

new weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 652
positions:
[ 0.04409202  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202
  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202
  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202
  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202
  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202  0.04409202]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.0

new weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 658
positions:
[ 0.04373513  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513
  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513
  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513
  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513
  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513  0.04373513]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.0

[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 664
positions:
[ 0.04338356  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356
  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356
  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356
  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356
  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356  0.04338356]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03

new weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333]
---------------------



PERIOD 670
positions:
[ 0.04374554  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554
  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554
  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554
  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554
  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554  0.04374554]
weights:
[ 0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333  0.03333333
  0.03333333  0.03333333  0.03333333  0.03333333  0.0

In [ ]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", ConstantRebalancer, Dummy, True)
s.setup_params(expert_args={"eta": -0.01})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01})
s.run(log=True, logpath="logs")

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EGRecent, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01, "n": 100})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01, "n": 750})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(expert_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))



In [ ]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(expert_args={})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run simulation where we each expert only recommends to buy when the price crosses 0.5 standard deviations below the 10-day moving average. 
## Otherwise doesn't do anything
## Sell position when price gets above 0.5 standard deviations below the 10-day MA

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01, "n": 1250},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Momentum, True)

years = 2. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Momentum, True)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Momentum, True)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.001, "n": 500},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
pd.__version__